# Notebook E-tivity 1 CE4021

Student name:Phil Clarke

Student ID: 23291567

<hr style="border:2px solid gray"> </hr>

## Task - DRAFT

Write 2 Python functions:

Function 1:
The first one to symbolically calculate the derivative of a polynomial with one variable. The input should be a polynomial and the output should also be a polynomial.

Function 2:
The second one to evaluate (i.e. get the numerical value) of a polynomial for a given value of its variable. The input should be a polynomial and a value (point at which to evaluate the polynomial). The output should be a scalar.
Test your code with a few salient polynomials (minimum of 3) for which you have calculated the derivative of these polynomials manually.

# Function 1

In [ ]:
#Function 1
#The first task is to a) understand a polynomial in Python from an input and calculate its derivative.
#Using dictionaries with parsing appears to be the best starting point.
#The output should be in the same format as the input.

#Creating an input for a polynomial
polynomial = input("Please enter a polynomial in the format '1+x^2-3x^7+100x^9' - Note it will not work for negative powers however order does not matter:") 
print(polynomial)
#FOR LATER - Error checking on neagtive powers?


In [ ]:
def polynomialparser(polynomial):
    # create a blank list to store the individial terms of the polynomial
    #set the initial term as "" for the purposes of iteration
    terms = []
    initial_term = ""

    # Iterate through each character in the expression creating terms everything we see a - or + sign
    for char in polynomial:
        if char in "+-":
            terms.append(initial_term)
            initial_term = char
        else:
            # If it's not a '+' or '-' sign, add the character to the current term
            initial_term += char

    # Add the last term to the list (assuming the expression ends with a term)
    terms.append(initial_term)

    # Filter out any empty strings
    #terms = [term for term in terms if term]

    # Initialize a dictionary to store coefficients and powers of the polynominal
    poly_dict = {}
    
    # Iterate through the terms
    for term in terms:
        coefficient = 1  # Default coefficient if none is specified for example "x or x^2"
        power = 0       # Default power if none is specified  for example "1 or 5 or n as a cooefficent"

        # Split the term into parts based on 'x', so therefore seperating out the coeff and the power parts
        parts = term.split('x')

        if parts[0]:  # Check if there's a coefficient part before 'x'
            coefficient_str = parts[0]
            if coefficient_str != '-':
                coefficient = int(coefficient_str)

        if len(parts) > 1:  # Check if there's a power part after 'x'
            if '^' in parts[1]:
                power = int(parts[1].split('^')[1])

        poly_dict[power] = coefficient

    return poly_dict

#This part of the codes takes the current dictionary and essentially runs the power rule of differentiation on each value and key pair.
#reducing the power by 1 and multiplying the value by the key.

Old_dict = polynomialparser(polynomial)

new_dict = {}

for key, value in Old_dict.items():
    new_key = key - 1
    new_value = key * value
    new_dict[new_key] = new_value

# Here the dictionary is sorted by the keys to ensure it is displayed in ascending order of power, the input does not require that order.
sorted_dict = dict(sorted(new_dict.items()))

polynomial_terms = []

for power, coefficient in sorted_dict.items():
    if coefficient != 0:
        term = f"{coefficient}x^{power}" if power > 1 else f"{coefficient}x" if power == 1 else str(coefficient)
        polynomial_terms.append(term)

#Here the terms are joined back together
Dervitative_expression = " + ".join(polynomial_terms)

print("The Original polynomial expression:", polynomial)
print("The Derivative polynomial expression:", Dervitative_expression)


# Function 2


Function 2: The second one to evaluate (i.e. get the numerical value) of a polynomial for a given value of its variable. The input should be a polynomial and a value (point at which to evaluate the polynomial). The output should be a scalar. Test your code with a few salient polynomials (minimum of 3) for which you have calculated the derivative of these polynomials manually.

(note: I take it that the code above can be recycled, and that I should for completeness, evaulate the polynomial and the dirative.)


In this first part I will make a continuation of the polynomial derivative tools above

In [75]:
print(Old_dict)
print(new_dict)

{0: 2, 5: 1}
{-1: 0, 4: 5}


In [92]:
def evaluation(dict,x):
        result = 0
        for power, coefficent in dict.items():
            result += coefficent * (x ** power)
        return result
x= 3
value_orig=evaluation(Old_dict, x)
value_derive=evaluation(new_dict, x)
print("x= ",x)
print("The Value of Original polynomial expression at x=:", value_orig)
print("The Derivative polynomial expressionat x=:", value_derive)


x=  3
The Value of Original polynomial expression at x=: 245
The Derivative polynomial expressionat x=: 405.0


# Testing

In this section, I have listed dictionary values of serval polynomials in the format {power: coefficent} pairs.  Each pair is multiplied by a value of z

In [101]:
polynomial_1={0: 2, 5: 1, 4:6, 2:-2}
print(evaluation(polynomial_1, 3))
polynomial_2={0: 6, 3: 1, 3:3, 2:-2}
print(evaluation(polynomial_2, 1000))
polynomial_3={0: 3, 2: 1, 4:8, 2:-2}
print(evaluation(polynomial_3, 3))
polynomial_4={0: 99, 7: 1, 8:9, 2:-2}
print(evaluation(polynomial_4, 99))
polynomial_5={0: 4, 8: 1, 4:2, 2:-2}
print(evaluation(polynomial_5, 2))
polynomial_6={0: 7, 7: 1, 6:1, 2:-2}
print(evaluation(polynomial_6, 1))
polynomial_7={0: 9, 6: 1, 4:2}
print(evaluation(polynomial_7, 0))


713
2998000006
633
83140229033284005
284
7
9


<hr style="border:2px solid gray"> </hr>

## Reflection

Write your reflection in below cell. With reference to the Rubric for E-tivity 1:
- Provide an accurate description of your code with advantages and disadvantages of design choices.
I went with a design choice based around dealing with the polynomial in the a dictionary and dealing with the different elements in terms of the power rule.  My first design (not here) was extermely manual and only worked with up to x^9, meaning as I manually entered what I wanted to happend depending on where the values were in the dictionary.  this version is more versitile as it ignores order and splits the value of the power and cooefficent.

Definitely the hardest element was the parsing, using regex this would have been a lot easier however this was not the exercise.

The derivation part, using the for loop on the key and value pairs, I think is an elegant solution to the problem as it does efficently does the differentation and keeps the value in a dictionary form.

Parsing back into the new polynomial was equally tricky as the parsing in comparsion.

The evaulation section of a polynomial, was again a for loop for a value of x summed, which worked well.  I have carried this out both connected to the main exercise (function 1) but also, left it to evulate from dictionary forms of polynomials for demonstration purposes.

- Compare your approach to alternative (peer) approaches.


- Clearly describe how you have used your peers' work/input and how this has affected your own understanding / insights.

Originally I went down the import re route but thanks to the forum I had that explained.  looking at some of the other solution I defintely went back and looked at my original solution to the problem as it was not good enough.  


If you have not used peer input, you may state this, but your submission history in Gitlab should clearly show this is the case.